In [11]:
import re

In [12]:
import pandas as pd
import bson
import pymongo
from tqdm import tqdm
tqdm.pandas()
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [13]:
from tqdm import tqdm
import urllib.request
from PIL import Image

In [14]:
def get_tags(_id):
    info = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})
    tags = list(set(info["bert_tags"] + info["entity_prepared"]))
    shifts = info["shifts_from_bert"]
    industries = info["industries_from_bert"]
    
    return tags, shifts, industries

In [15]:
# Import FPDF class
from fpdf import FPDF
 
# Import glob module to find all the files matching a pattern
import glob
 
# Image extensions
# image_extensions = ("*.png", "*.jpg", "*.gif")
 
image_extensions = ("daily-shot/*.png", "daily-shot/*.jpg", "daily-shot/*.gif")    
# This list will hold the images file names
# images = ["more-landing-page-examples.jpg",
#           "Everyday-Examples-of-Artificial-Intelligence-and-Machine-Learning-950x540.jpg"]
images = []
 
# Build the image list by merging the glob results (a list of files)
# for each extension. We are taking images from current folder.
for extension in image_extensions:
    images.extend(glob.glob(extension))
    
# Create instance of FPDF class
pdf=FPDF('P','in','letter')
# Add new page. Without this you cannot create the document.
pdf.add_page()
# Set font to Arial, 'B'old, 16 pts
pdf.set_font('Arial','B',16.0)
 
# Page header
pdf.cell(4.0,1.0,'Images in this folder')
pdf.ln(0.25)
 
# Smaller font for image captions
pdf.set_font('Arial','',10.0)
 
# Loop through the image list and position
# them with their caption one below the other


for image in images[:100]:
    
    _id = re.sub("daily-shot/", "", image)
    _id = re.sub(".png|.jpg", "", _id)
    tags, shifts, industries = get_tags(_id)

    # Setting image width to half the page and
    # height to 1/4th of the page
    cover = Image.open(image)
    width, height = cover.size
    ratio = width/height
    
    imageH = pdf.h/4.0
    imageW = imageH * ratio
    
    
#     pdf.cell(3.0, 0.0, image)
#     pdf.ln(0.15)
    
    pdf.image(image, w=imageW, h=imageH)
    pdf.ln(0.25)
 
    # Image caption
    pdf.cell(3.0, 0.0, "ID: " + _id)
    pdf.ln(0.35)
    
    
    pdf.cell(3.0, 0.0, "TAGS: " + str(tags))
    pdf.ln(0.35)
    
    pdf.cell(3.0, 0.0, "SHIFTS: "+ str(shifts))
    pdf.ln(0.35)
    
    pdf.cell(3.0, 0.0, "INDUSTRIES: "+ str(industries))
    pdf.ln(0.35)
    
    pdf.cell(3.0, 0.0, "")
    pdf.ln(0.35)
    
# output content into a file ('F')
pdf.output('daily-shot.pdf','F')

''